# Risk Profotlio Management
identify factor which are driving fortfolio  returns

Investment Risk
- measure of uncertainty of future returns (dispersion or variance of financial returns)

Percentage Change and log percentage change

The Adjusted Close column is the most important. It is normalized for stock splits, dividends, and other corporate actions, and is a true reflection of the return of the stock over time. 

## Univariate Investment Risk & Returns

In [ ]:
# Import pandas as pd
import pandas as pd

# Read in the csv file and parse dates
StockPrices = pd.read_csv(fpath_csv, parse_dates=['Date'])

# Ensure the prices are sorted by Date
StockPrices = StockPrices.sort_values(by='Date')

# Calculate the daily returns of the adjusted close price
StockPrices['Returns'] = StockPrices['Adjusted'].pct_change()

# Plot the returns column over time
StockPrices['Returns'].plot()
plt.show()

# Convert the decimal returns into percentage returns
percent_return = StockPrices['Returns']*100

# Drop the missing values
returns_plot = percent_return.dropna()

# Plot the returns histogram
plt.hist(returns_plot, bins=75)
plt.show()

Standard Deviation (voltality)'
- an investement with higer standard deviation (voltality) is viewed as higher risk investment 
- scalling voltality, 
    - $\sigma_{annual}$  = $ \sigma_{daily} \times \sqrt {252}$,  
    - $\sigma_{monthly}$  = $ \sigma_{daily} \times  \sqrt {21}$

Average annualised return = $(1+\mu)^{252}-1$

In [ ]:
# Import numpy as np
import numpy as np

# Calculate the average daily return of the stock
mean_return_daily = np.mean((StockPrices['Returns']))
print(mean_return_daily)

# Calculate the implied annualized average return
mean_return_annualized = ((1+mean_return_daily)**252)-1
print(mean_return_annualized)

# Calculate the standard deviation of daily return of the stock
sigma_daily = np.std(StockPrices['Returns'])
print(sigma_daily)

# Calculate the daily variance
variance_daily = sigma_daily**2
print(variance_daily)

# Annualize the standard deviation
sigma_annualized = sigma_daily*np.sqrt(252)
print(sigma_annualized)

# Calculate the annualized variance
variance_annualized = sigma_annualized**2
print(variance_annualized)

- skew - wether it's left-leaning(positive) or right-leaning(negative). in finance we prefer left-leaning positive skew because the loss are predictable with long right distribution for high returns.
- Kurtosis is a measure of how thick the tail of a distribution 
- most financial returns are leptokratic (when  a distribution has a postive excess kurtosis (higher than 3)) normal distribution has a kurtosis of 3
- excess kurtosis = kurtosis - 3 indicates higher risk (outliers, the far right & the far left, are more common - relatively higher probability than a normal distribution)

Testing for normality - Shapiro-Wilk test (null hypothesis - the ditribution is normal) so if p-values is less thann -0.05 we can safely assume that the distibution is non-normal. 


In [ ]:
# Import skew from scipy.stats
from scipy.stats import skew

# Drop the missing values
clean_returns = StockPrices["Returns"].dropna()

# Calculate the third moment (skewness) of the returns distribution
returns_skewness = skew(clean_returns)
print(returns_skewness)

# Import kurtosis from scipy.stats
from scipy.stats import kurtosis

# Calculate the excess kurtosis of the returns distribution
excess_kurtosis = kurtosis(clean_returns)
print(excess_kurtosis)

# Derive the true fourth moment of the returns distribution
fourth_moment = excess_kurtosis + 3
print(fourth_moment)

## Portfolio Composition & Backstasting

Portfolio refers to a bundle of different stocks each with different weight.
- $R_p $ , Portfolio return
- $R_{a_n} $, Asset n Return
- $w_{a_n}$, Weight for asset n
$$ R_p = R_{a_1}w_{a_1} + R_{a_2}w_{a_2} + \dots + R_{a_n}w_{a_n}$$

In [ ]:
# Finish defining the portfolio weights as a numpy array
portfolio_weights = np.array([0.12, 0.15, 0.08, 0.05, 0.09, 0.10, 0.11, 0.14, 0.16])

# Calculate the weighted stock returns
WeightedReturns = StockReturns.mul(portfolio_weights, axis=1)

# Calculate the portfolio returns
StockReturns['Portfolio'] = WeightedReturns.sum(axis=1)

# Plot the cumulative portfolio returns over time
CumulativeReturns = ((1+StockReturns["Portfolio"]).cumprod()-1)
CumulativeReturns.plot()
plt.show()

# How many stocks are in your portfolio?
numstocks = 9

# Create an array of equal weights across all assets
portfolio_weights_ew = np.repeat(1/numstocks, numstocks)

# Calculate the equally-weighted portfolio returns
StockReturns['Portfolio_EW'] = StockReturns.iloc[:, :9].mul(portfolio_weights_ew, axis=1).sum(axis=1)
cumulative_returns_plot(['Portfolio', 'Portfolio_EW'])

# Create an array of market capitalizations (in billions)
market_capitalizations = np.array([601.51, 469.25, 349.5, 310.48, 299.77, 356.94, 268.88, 331.57, 246.09])

# Calculate the market cap weights
mcap_weights = market_capitalizations/sum(market_capitalizations)

# Calculate the market cap weighted portfolio returns
StockReturns['Portfolio_MCap'] = StockReturns.iloc[:, 0:9].mul(mcap_weights, axis=1).sum(axis=1)
cumulative_returns_plot(['Portfolio', 'Portfolio_EW', 'Portfolio_MCap'])

## Correlation & Covariance 

- Finding a linear correlation between two stocks

$ \mathrm{Cov}(A,B) = Var(A) + Var(B) + 2Var(A)Var(B)$ Covariance
When dealing with multiple stocks, we can deal with covariance matrix 

Protfolio standart deviation
- $ \sigma_{Portfolio}^2 $  Portfolio variance
- $ w $ weights vector
- $ \sum $ covariance matrix 
$$ \sigma_{Portfolio} = \sqrt{w_T \cdot  \sum  \cdot w}$$

In [ ]:
# Calculate the correlation matrix
correlation_matrix = StockReturns.corr()

# Print the correlation matrix
print(correlation_matrix)

# Import seaborn as sns
import seaborn as sns

# Create a heatmap
sns.heatmap(correlation_matrix,
            annot=True,
            cmap="YlGnBu", 
            linewidths=0.3,
            annot_kws={"size": 8})

# Plot aesthetics
plt.xticks(rotation=90)
plt.yticks(rotation=0) 
plt.show()

# Calculate the covariance matrix
cov_mat = StockReturns.cov()

# Annualize the co-variance matrix
cov_mat_annual = cov_mat*(252) # no need to sqrt 252 because it's a variance not standard deviation

# Print the annualized co-variance matrix
print(cov_mat_annual)

# Import numpy as np
import numpy as np

# Calculate the portfolio standard deviation
portfolio_volatility = np.sqrt(np.dot(portfolio_weights.T, np.dot(cov_mat_annual, portfolio_weights)))
print(portfolio_volatility)

## Markowitz Portfolio

sharpe ratio is a measure of risk adjusted return

$$ S = \frac {R_{\alpha} - r_{f}}{\sigma_{\alpha}}$$
- S, sharpe ratio
- $ R_{\alpha} $ Asset return
- $ r_{f} $ risk-free Asset return
- $ \sigma_{\alpha} $ Asset volatilty

how much return an investor can expect for each incremental unit of risk and can be used to compare different portfolio with different risk

The *maximum Sharpe ratio*, or *MSR portfolio*, which lies at the apex of the efficient frontier, can be constructed by looking for the portfolio with the highest Sharpe ratio.

Unfortunately, the MSR portfolio is often quite erratic. Even though the portfolio had a high historical Sharpe ratio, it doesn't guarantee that the portfolio will have a good Sharpe ratio moving forward.

In [ ]:
# Risk free rate
risk_free = 0

# Calculate the Sharpe Ratio for each asset
#annualized rate of return and annualized standard deviation
RandomPortfolios['Sharpe'] = (RandomPortfolios["Returns"] - risk_free)/RandomPortfolios["Volatility"]


# Print the range of Sharpe ratios
print(RandomPortfolios['Sharpe'].describe()[['min', 'max']])

RandomPortfolios.head()

# Sort the portfolios by Sharpe ratio
sorted_portfolios = RandomPortfolios.sort_values(by=['Sharpe'], ascending=False)

# Extract the corresponding weights
MSR_weights = sorted_portfolios.iloc[0, 0:numstocks]

# Cast the MSR weights as a numpy array
MSR_weights_array = np.array(MSR_weights)

# Calculate the MSR portfolio returns
StockReturns['Portfolio_MSR'] = StockReturns.iloc[:, 0:numstocks].mul(MSR_weights_array, axis=1).sum(axis=1)

# Plot the cumulative returns
cumulative_returns_plot(['Portfolio_EW', 'Portfolio_MCap', 'Portfolio_MSR'])


# Sort the portfolios by volatility
sorted_portfolios = RandomPortfolios.sort_values(by=['Volatility'], ascending=True)

# Extract the corresponding weights
GMV_weights = sorted_portfolios.iloc[0, 0:numstocks]

# Cast the GMV weights as a numpy array
GMV_weights_array = np.array(GMV_weights)

# Calculate the GMV portfolio returns
StockReturns['Portfolio_GMV'] = StockReturns.iloc[:, 0:numstocks].mul(GMV_weights_array, axis=1).sum(axis=1)

# Plot the cumulative returns
cumulative_returns_plot(['Portfolio_EW', 'Portfolio_MCap', 'Portfolio_MSR', 'Portfolio_GMV'])

## Capital Asset Pricing Model (CAPM)

excess rate = return rate - risk-free return 

$$ E(R_p) - RF = \beta_p(E(R_b)- RF)$$

- $ R_p - RF$ , excess rate of expected rate of return of portfolio P
- $ R_b $ RF , excess rate of expected rate of return of the broad market portfolio B
- the reginol risk free rate

- $ \beta_p $ beta p or exposure to the broad market portfolio B

the higher the beta, the higher the exposure to the broad market portfolio B

Calculate historical beta using co-variance 

$$ \beta_p = \frac{Cov(R_p, R_b)}{Var(R_b)}$$

- $Cov(R_p, R_b)$ co-variance between portfolio P  and  bencmark maket index b
- $ Var(R_b)$ the variance of the benchmark market index B

In [ ]:
# Calculate excess portfolio returns
FamaFrenchData['Portfolio_Excess'] = FamaFrenchData["Portfolio"] - FamaFrenchData["RF"]

# Plot returns vs excess returns
CumulativeReturns = ((1+FamaFrenchData[['Portfolio','Portfolio_Excess']]).cumprod()-1)
CumulativeReturns.plot()
plt.show()

FamaFrenchData.head()

# Calculate the co-variance matrix between Portfolio_Excess and Market_Excess
covariance_matrix = FamaFrenchData[['Portfolio_Excess', 'Market_Excess']].cov()

# Extract the co-variance co-efficient
covariance_coefficient = covariance_matrix.iloc[0, 1]
print(covariance_coefficient)

# Calculate the benchmark variance
benchmark_variance = FamaFrenchData['Market_Excess'].var()
print(benchmark_variance)

# Calculating the portfolio market beta
portfolio_beta = covariance_coefficient/benchmark_variance
print(portfolio_beta)


import statsmodels.formula.api as smf

# Define the regression formula
CAPM_model = smf.ols(formula='Portfolio_Excess ~ Market_Excess', data=FamaFrenchData)

# Print adjusted r-squared of the fitted regression
CAPM_fit = CAPM_model.fit()
print(CAPM_fit.rsquared_adj)

# Extract the beta
regression_beta = CAPM_fit.params["Market_Excess"]
print(regression_beta)

## portfolio interpretaion
Your portfolio beta is 0.9738. You can think of market beta as a measure of your exposure to the broad stock market. For every 1.0% rise (or fall) in the market, you can expect your portfolio to rise (fall) roughly 0.97%.

The adjusted r-squared is 0.7943. A high adjusted r-squared (close to 1) means that the majority of your portfolio's movements can be explained by the factors in your model.


The Fama-French 3 Factors model
$$ R_p  = RF+ \beta_b(R_b- RF) + b_{SMB} \cdot SMB + b_{HML} \cdot HML + \alpha$$

- SMB, small minus big
- HML high minus low factor
- b, beta for both SMB and HML are the exposure of the portolio to the HML factor and SMB factor
- alpha is an error term. in the efficient market, alpha is equivalent to zero


The HML factor is constructed by calculating the return of growth stocks, or stocks with high valuations, versus the return of value stocks.
The SMB factor is constructed by calculating the return of small-cap stocks, or stocks with small market capitalizations, versus the return of large-cap stocks.

In [ ]:
# Import statsmodels.formula.api
import statsmodels.formula.api as smf 

# Define the regression formula
FamaFrench_model = smf.ols(formula='Portfolio_Excess ~ Market_Excess + SMB + HML ', data=FamaFrenchData)

# Fit the regression
FamaFrench_fit = FamaFrench_model.fit()

# Extract the adjusted r-squared
regression_adj_rsq = FamaFrench_fit.rsquared_adj
print(regression_adj_rsq)

# Extract the p-value of the SMB factor
smb_pval = FamaFrench_fit.pvalues["SMB"]

# If the p-value is significant, print significant
if smb_pval < 0.05:
    significant_msg = 'significant'
else:
    significant_msg = 'not significant'

# Print the SMB coefficient
smb_coeff = FamaFrench_fit.params["SMB"]
print("The SMB coefficient is ", smb_coeff, " and is ", significant_msg)

## Estimating Tail Risk

### Historical Drawdown
the percentage loss from the highest cumulative historical point 

$$ D = \frac{r_t}{RM} - 1$$

- D, drawdown
- $ r_t $ cumulative return at time t
- RM Running Maximum 

### Value at Risk
there is quantile attached to it. for example if 95-th percentile is choosen. at the worst 5% of the trading days, the investement will loss more than x percentage/b with 95 cenrtainty, the loss will not excedd x percentage in a given day based on historical data. 


In [ ]:
# Calculate the running maximum
running_max = np.maximum.accumulate(cum_rets)

# Ensure the value never drops below 1
running_max[running_max < 1] = 1

# Calculate the percentage drawdown
drawdown = (cum_rets)/running_max - 1

# Plot the results
drawdown.plot()
plt.show()

# Calculate historical VaR(95)
var_95 = np.percentile(StockReturns_perc, 5)
print(var_95)

# Sort the returns for plotting
sorted_rets = StockReturns_perc.sort_values()

# Plot the probability of each sorted return quantile
plt.hist(sorted_rets, density=True, stacked=True)

# Denote the VaR 95 quantile
plt.axvline(x=var_95, color='r', linestyle='-', label="VaR 95: {0:.2f}%".format(var_95))
plt.show()

# Historical CVaR 95
cvar_95 = StockReturns_perc[StockReturns_perc <  var_95].mean()
print(cvar_95)

# Sort the returns for plotting
sorted_rets = sorted(StockReturns_perc)

# Plot the probability of each return quantile
plt.hist(sorted_rets, density=True, stacked=True)

# Denote the VaR 95 and CVaR 95 quantiles
plt.axvline(x=var_95, color="r", linestyle="-", label='VaR 95: {0:.2f}%'.format(var_95))
plt.axvline(x=cvar_95, color='b', linestyle='-', label='CVaR 95: {0:.2f}%'.format(cvar_95))
plt.show()

Value at Risk can also be computed parametrically using a method known as variance/co-variance VaR. This method allows you to simulate a range of possibilities based on historical return distribution properties rather than actual return values. You can calculate the parametric VaR(90) using:

In [ ]:
# Import norm from scipy.stats
from scipy.stats import norm

# Estimate the average daily return
mu = np.mean((StockReturns))

# Estimate the daily volatility
vol = np.std((StockReturns))

# Set the VaR confidence level
confidence_level = 0.05

# Calculate Parametric VaR
var_95 = norm.ppf(confidence_level, mu, vol)
print('Mean: ', str(mu), '\nVolatility: ', str(vol), '\nVaR(95): ', str(var_95))

# Aggregate forecasted VaR
forecasted_values = np.empty([100, 2])

# Loop through each forecast period
for i in range(100):
    # Save the time horizon i
    forecasted_values[i, 0] = i
    # Save the forecasted VaR 95
    forecasted_values[i, 1] = var_95*np.sqrt(i+1)
    
# Plot the results
plot_var_scale()

## monte carlo simulation

to create thousands of possible casses. we can analyse each one of them just like we did before - looking at VAr & CVar

In [ ]:
# Set the simulation parameters
mu = np.mean(StockReturns)
vol = np.std(StockReturns)
T = 252
S0 = 10

# Add one to the random returns
rand_rets = np.random.normal(mu, vol, T) + 1

# Forecasted random walk
forecasted_values = rand_rets.cumprod() * S0

# Plot the random walk
plt.plot(range(0, T), forecasted_values)
plt.show()

# Loop through 100 simulations
for i in range(100):

    # Generate the random returns
    rand_rets = np.random.normal(mu, vol, T) + 1
    
    # Create the Monte carlo path
    forecasted_values = S0*(rand_rets).cumprod()
    
    # Plot the Monte Carlo path
    plt.plot(range(T), forecasted_values)

# Show the simulations
plt.show()


# Aggregate the returns
sim_returns = []

# Loop through 100 simulations
for i in range(100):

    # Generate the Random Walk
    rand_rets = np.random.normal(mu, vol, T)
    
    # Save the results
    sim_returns.append(rand_rets)

# Calculate the VaR(99)
var_99 = np.percentile(sim_returns, 1)
print("Parametric VaR(99): ", round(100*var_99, 2),"%")

- Moments and Distibutions
- Portfolio composition
- Correlation & Covariance
- Markowitz optimisation
- Beta & CAPM
- FAMA French Factor Modelling
- ALpha - term that measures the efficient market
- Var & CVar
- Monte Carlo Simulation


## Covariance Matrix

- **Diagonal entries** are variance of each individual asset
- **off-diagonal entries** are covariance between two different asset depending on which cell we are looking at

In [ ]:
# Calculate the 30-day rolling window of portfolio returns
returns_windowed = portfolio_returns.rolling(20)

# Compute the annualized volatility series
volatility_series = returns_windowed.std()*np.sqrt(252)

# Plot the portfolio volatility
volatility_series.plot().set_ylabel("Annualized Volatility, 30-day Window")
plt.show()

: 

## Risk factors and financial crisis

### Volatility 
- measure of dispersion of returns around expected value
### Expected return
- sample mean in a time-series context

What drives expected returns and volatility?
- the drives are known as risk factors: events and variables driving portfolio returns and volatility

#### Systematic Risk
risk factor(s) affecting volatility of all portfolio assets
one example is market risk; systematic risk from general financial market movements 

example of financial systemic risk
- inflation, interest rate changes, economic climate changes

#### Idiosyncratic Risk
- risk spesific to a particular class/asset class
example- bond asset : issuer risk and default, firm/sector characterisation: market cap, sector shocks, book-to-market value ratio

In [ ]:
# Convert daily returns to quarterly average returns
returns_q = returns.resample('Q').mean()

# Examine the beginning of the quarterly series
print(returns_q.head())

# Now convert daily returns to weekly minimum returns
returns_w = returns.resample('W').min()

# Examine the beginning of the weekly series
print(returns_w.head)

# Add a constant to the regression
mort_del = sm.add_constant(mort_del)

# Create the regression factor model and fit it to the data
results = sm.OLS(port_q_mean, mort_del).fit()

# Print a summary of the results
print(results.summary())

## Modern Portfolio Theory

- aim is to always maximise returns given a risk level
- one way to do this by adjusting the weights of each asset in portfolio. the method is called efficient portfolio theory - portfolio with weights generating the highest expected return given a level of risk.
- weights and expected returns can be visualised on efficient frontier plots 

Portfolio optimization relies upon an *unbiased* and *efficient* estimate of asset covariance. Although sample covariance is unbiased, it is not efficient--extreme events tend to be overweighted.

One approach to alleviate this is through "covariance shrinkage", where large errors are reduced ('shrunk') to improve efficiency. In this exercise, you'll use pypfopt.risk_models's CovarianceShrinkage object to transform sample covariance into an efficient estimate. The textbook error shrinkage method, .ledoit_wolf(), is a method of this object.

Asset prices are available in your workspace. Note that although the CovarianceShrinkage object takes prices as input, it actually calculates the covariance matrix of asset returns, not prices.

In [ ]:
# Load the investment portfolio price data into the price variable.
prices = pd.read_csv("portfolio.csv")

# Convert the 'Date' column to a datetime index
prices['Date'] = pd.to_datetime(prices['Date'], format='%d/%m/%Y')
prices.set_index(['Date'], inplace = True)

# Import the mean_historical_return method
from pypfopt.expected_returns import mean_historical_return

# Compute the annualized average historical return
mean_returns = expected_returns(prices, frequency = 252)

# Plot the annualized average historical return
plt.plot(mean_returns, linestyle = 'None', marker = 'o')
plt.show()

# Import the CovarianceShrinkage object
from pypfopt.risk_models import CovarianceShrinkage

# Create the CovarianceShrinkage instance variable
cs = CovarianceShrinkage(prices)

# Compute the sample covariance matrix of returns
sample_cov = prices.pct_change().cov() * 252

# Compute the efficient covariance matrix of returns
e_cov = cs.ledoit_wolf()

# Display both the sample covariance_matrix and the efficient e_cov estimate
print("Sample Covariance Matrix\n", sample_cov, "\n")
print("Efficient Covariance Matrix\n", e_cov, "\n")

# Create a dictionary of time periods (or 'epochs')
epochs = { 'before' : {'start': '1-1-2005', 'end': '31-12-2006'},
           'during' : {'start': '1-1-2007', 'end': '31-12-2008'},
           'after'  : {'start': '1-1-2009', 'end': '31-12-2010'}
         }

# Compute the efficient covariance for each epoch
e_cov = {}
for x in epochs.keys():
  sub_price = prices.loc[epochs[x]['start']:epochs[x]['end']]
  e_cov[x] = CovarianceShrinkage(sub_price).ledoit_wolf()

# Display the efficient covariance matrices for all epochs
print("Efficient Covariance Matrices\n", e_cov)

# Initialize the Crtical Line Algorithm object
efficient_portfolio_during = CLA(returns_during, ecov_during)

# Find the minimum volatility portfolio weights and display them - $min_volatility is a method
print(efficient_portfolio_during.min_volatility())

# Compute the efficient frontier
(ret, vol, weights) = efficient_portfolio_during.efficient_frontier() #efficint_frontier()

# Add the frontier to the plot showing the 'before' and 'after' frontiers
plt.scatter(vol, ret, s = 4, c = 'g', marker = '.', label = 'During')
plt.legend()
plt.show()

## Deciding between options
- wether or not to pay extra to get refunable booking made if anything happes.

1. chance of negative shock (probability of loss) 
2. loss associated with shocks - Var, CVar'
3. Desire to avoid shock (personal feeling )- risk tolerance

In [ ]:
# Import the Student's t-distribution
from scipy.stats import t

# Create rolling window parameter list
mu = losses.rolling(30).mean()
sigma = losses.rolling(30).std()
rolling_parameters = [(29, mu[i], s) for i,s in enumerate(sigma)]

# Compute the 99% VaR array using the rolling window parameters
VaR_99 = np.array( [ t.ppf(0.99, *params) 
                    for params in  rolling_parameters] )

# Plot the minimum risk exposure over the 2005-2010 time period
plt.plot(losses.index, 0.01 * VaR_99 * 100000)
plt.show()